In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

import kelp_compute

sp.init_printing()
%matplotlib inline

from fortran_wrappers.pykelp3d_wrap import pykelp3d_wrap as f90
f90.solve_rte_with_callbacks?

In [3]:
space = sp.var('x, y, z')
x, y, z = space
vec_x = sp.Matrix(space)

angle = sp.var(r'theta, phi')
th, ph = angle
vec_om = sp.Matrix([sp.sin(ph)*sp.cos(th), sp.sin(ph)*sp.sin(th), sp.cos(ph)])

angle_prime = sp.var(r'theta_p, phi_p')
thp, php = angle_prime
vec_omp = sp.Matrix([sp.sin(php)*sp.cos(thp), sp.sin(php)*sp.sin(thp), sp.cos(php)])

In [4]:
def grad(expr, space=space):
    return sp.Matrix([sp.diff(expr, d) for d in space])

In [5]:
def sphere_integral(expr, angle):
    theta, phi = angle
    return sp.integrate(expr*sp.sin(phi), (theta, 0, 2*sp.pi), (phi, 0, sp.pi))

In [6]:
def dot(a, b):
    return sum(a.T * b)

---

In [7]:
def calculate_source(L, b, a, beta, params=()):
    L_om = L(*space, *angle, *params)
    L_omp = L(*space, *angle_prime, *params)
    
    deriv = dot(vec_om, grad(L_om))
    atten = (a(*space)+b)*L_om
    
    scat_integrand = dot(vec_om, grad(L_omp))
    scat = sphere_integral(scat_integrand, angle=angle_prime)
    
    source = deriv + atten - scat
    
    return source

In [8]:
def calculate_bc(L, params=()):
    z = space[-1]
    zmin = 0
    return L(*space, *angle, *params).subs(z, zmin)

In [9]:
b = sp.var('b')
params = sp.var('alpha, gamma')

In [10]:
def uniform_vsf(delta):
    return 1/(4*sp.pi)

In [11]:
# TODO: No upwelling light from bottom
# Also, spatially homogeneous BC
def prod_L(x, y, z, theta, phi, alpha, gamma):
    return (
        (
            (
                (2+sp.sin(2*sp.pi*x/alpha))
                *(2+sp.sin(2*sp.pi*y/alpha))
                *(sp.sin(2*sp.pi*z/gamma))
            ) + sp.exp(-z/gamma)
        )
        *(2+sp.sin(phi))
    )

In [12]:
# TODO: Should be periodic on correct grid
def prod_a(x, y, z):
    return (2+sp.sin(2*sp.pi*x))*(2+sp.sin(2*sp.pi*y))*(1+sp.tanh(z-gamma))

In [13]:
prod_source = calculate_source(prod_L, b, prod_a, uniform_vsf, params)
prod_bc = calculate_bc(prod_L, params)

In [19]:
alpha = 1
gamma = 1
b = 0.0

In [23]:
prod_source_sym = sp.lambdify(
    (*space, *angle), 
    prod_source.subs('alpha', alpha).subs('gamma', gamma).subs('b', b),
    modules=("sympy",)
)

prod_a_sym = sp.lambdify(
    space,
    prod_a(*space),
    modules=("sympy",)
)

prod_bc_sym = sp.lambdify(
    angle, 
    prod_bc.subs('alpha', alpha).subs('gamma', gamma),
    modules=("sympy",)
)

uniform_vsf_sym = sp.lambdify(
    (theta,), 
    uniform_vsf(theta),
    modules=("sympy",)
)

# Convert callbacks to numpy functions
abs_func_N = np.vectorize(
    sp.lambdify(
        space,
        abs_func(*space),
        modules=("numpy",)
    )
)

source_func_N = np.vectorize(
    sp.lambdify(
        (*space, *angle),
        source_func(*space, *angle),
        modules=("numpy",)
    )
)


bc_func_N = np.vectorize(
    sp.lambdify(
        angle,
        bc_func(*angle),
        modules=("numpy",)
    )
)

vsf_func_N = np.vectorize(
    sp.lambdify(
        (theta,),
        vsf_func(theta),
        modules=("numpy",)
    )
)

In [24]:
abs_func = prod_a_sym
source_func = prod_source_sym
bc_func = prod_bc_sym
vsf_func = uniform_vsf_sym

In [22]:
ns = 6
nz = 6
na = 6
rope_spacing = 1
zmax = 1

lis_opts = "-i gmres"
fd_flag = False
num_scatters = 0

scalar_params, results = kelp_compute.solve_rte_with_callbacks_full(
    ns, nz, na,
    rope_spacing, zmax,
    b, abs_func, source_func, bc_func, vsf_func,
    num_scatters, fd_flag, lis_opts
)

In [161]:
def gen_grid(ns, nz, na, rope_spacing, zmax):
    ds = rope_spacing/ns
    dz = zmax/nz
    
    x = y = -rope_spacing/2 + ds * (np.arange(ns) + 1/2)
    z = dz * (np.arange(nz) + 1/2)
    
    ntheta = nphi = na
    nomega = ntheta*(nphi-2) + 2
    
    dtheta = 2*np.pi/ntheta
    dphi = np.pi/(nphi-1)
    
    theta = dtheta * np.arange(ntheta)
    phi = dphi * np.arange(nphi)
    
    #  function phat(angles, l, m) result(p)
    #    class(angle2d) :: angles
    #    integer l, m, p
    #
    #    if(m .eq. 1) then
    #       p = 1
    #    else if(m .eq. angles%nphi) then
    #       p = angles%nomega
    #    else
    #       p = (m-2)*angles%ntheta + l + 1
    #    end if
    #  end function phat

    l = np.arange(ntheta)
    m = np.arange(nphi)
    p = np.arange(nomega)

    L, M = np.meshgrid(l, m, indexing='ij')
    
    phat = (M-1)*ntheta + L + 1
    phat[:,0] = 0
    phat[:,-1] = nomega-1
    
    theta_p = np.zeros(nomega)
    phi_p = np.zeros(nomega)
    theta_p[phat] = theta[L]
    theta_p[0] = theta_p[-1] = 0
    phi_p[phat] = phi[M]
    
    # A bit redundant, but seems to work
    X, Y, Z, Theta = np.meshgrid(x, y, z, theta_p, indexing='ij')
    _, _, _, Phi = np.meshgrid(x, y, z, phi_p, indexing='ij')
    
    return X, Y, Z, Theta, Phi

In [18]:
results['rad'] 

array([[[[ 3.94586255e+01,  3.13867057e+01,  2.84925379e+01, ...,
          -5.15710079e+01,  2.53561824e+01, -4.45727941e+01],
         [ 7.73574772e+01,  7.69926201e+01,  6.50873245e+01, ...,
          -6.62754155e+01, -8.05771368e+00, -4.79466343e+01],
         [ 4.54623694e+01,  3.77176852e+01,  4.56866156e+01, ...,
          -6.82426736e+01, -3.85768200e+01, -5.18151496e+01],
         [-1.70252218e+01, -3.32794610e+01, -4.84200061e+01, ...,
          -6.65620436e+01, -5.58930314e+01, -5.62508695e+01],
         [-4.03790470e+01, -5.68656403e+01, -1.00159976e+02, ...,
          -7.37513549e+01, -5.26598355e+01, -6.13369580e+01],
         [ 2.38342393e+00,  5.42026558e+00, -1.62060285e+01, ...,
          -4.26563231e+01, -3.50721198e+01, -3.35037981e+01]],

        [[ 2.84921806e+01,  2.33068341e+01,  2.59537996e+01, ...,
          -1.37283482e+01,  9.52740425e+01, -4.28670526e+01],
         [ 5.70650382e+01,  5.76564125e+01,  5.32473973e+01, ...,
          -2.59058880e+01,  6.555626